#### Versão Online:
[Evolução Diferencial vs Jade vs Algoritmo Genético](https://mizerkowski.com.br/evolucao-diferencial-algoritmo-genetico-jade/)

In [1]:
import numpy as np
from scipy.optimize import differential_evolution
from differential_evolution import DifferentialEvolution
from genetic_algorithm import GeneticAlgorithm
from jade import JADifferentialEvolution
from problemas import PressureVessel, TensionCompressionSpring, SpeedReducer
import pandas as pd

In [2]:
def run(algorithm, problem, N, G, config):
    res = dict()
    for case in config:
        res[case] = []
        for i in range(N):
            alg = algorithm(problem.problem, problem.bounds, seed=i, G=G, **config[case])
            alg.run()
            res[case].append(alg.fun)
    return res

In [3]:
class ScipyAdapter:
    def __init__(self, problem, bounds, seed, G, mutation, recombination):
        self.func = differential_evolution
        self.result = None
        self.problem = problem
        self.bounds = bounds
        self.seed = seed
        self.maxiter = G
        self.mutation = mutation
        self.recombination = recombination

    @property
    def fun(self):
        return self.result.fun
        
    def run(self):
        self.result = self.func(self.problem, bounds=self.bounds, seed=self.seed, maxiter=self.maxiter, mutation=self.mutation, recombination=self.recombination)


In [4]:
N = 30
G = 200

In [5]:
config = {'case1':{'mutation':0.7, 'recombination':0.7},
          'case2':{'mutation':0.6, 'recombination':0.8},
          'case3':{'mutation':0.8, 'recombination':0.6}}

# 4.3.1 Example 8: Design of Pressure Vessel

A minimização do custo de material e produção de um vaso de pressão, este apresentando com 4 dimensões (grossura do casco, grossura da cabeça, radio interno e comprimento do cilindro) para serem adequadas pelo algoritmo. Estando a minimização sujeita a 4 desigualdades.

Os resultados da evolução diferencial e sua variação jade foram os que melhor funcionaram. Enquanto que o algoritmo genético não foi capaz de entrar em uma região de mínimo.

In [19]:
best_x = (0.8125, 0.4375, 42.098446, 176.636596)
pv = PressureVessel()
pv.pressure_vessel(best_x)-pv.constrains(best_x) # = 6059.714339

6059.714406596527

In [7]:
res_de = run(algorithm=DifferentialEvolution, problem=pv, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=pv, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=pv, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=pv, N=N, G=G, config=config)

In [8]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   6129.063891  6344.126139  6092.374045
std     123.364782   291.980889    82.679927
min    6059.932671  6059.733873  6059.731881
25%    6061.250665  6091.043242  6061.021478
50%    6091.151410  6371.054639  6063.480757
75%    6093.265213  6413.496441  6091.707510
max    6419.882870  6824.338635  6411.791999

DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   6061.783193  6059.729713  6081.522931
std       2.771037     0.020929    15.729263
min    6059.899275  6059.714788  6060.252735
25%    6060.231963  6059.717626  6071.956076
50%    6061.043693  6059.722702  6077.537347
75%    6062.335561  6059.730658  6085.606400
max    6074.836493  6059.807144  6119.812004

GA
               case1          case2          case3
count      30.000000      30.000000      30.000000
mean   144982.730449  144813.270168  107247.740253
std     76191.438607

# 4.3.3 Example 10: Design of Tension/Compression Spring

Sujeita a 4 desigualdades e possuindo 3 dimensões (diâmetro do fio, diâmetro média da mola e número de espiras), o problema é a minimização do peso de uma mola de tensão/compressão. A minimização de um vaso de pressão, com 4 dimensões (grossura do casco, grossura da cabeça, radio interno e comprimento do cilindro) e cuja função a ser minimizada representa o custo do material e da manufatura do mesmo. Estando a minimização sujeita a 4 desigualdades.

Todos os três casos da JADE chegaram aos menores valores da função encontrados na literatura, enquanto que a evolução diferencial teve um menor desvio padrão e valores mínimos próximos aos da literatura.

In [9]:
best_x = (0.051749, 0.358179, 11.203763)
tcs = TensionCompressionSpring()
tcs.tension_compression_spring(best_x)-tcs.constrains(best_x)

0.012664899963644788

In [10]:
res_de = run(algorithm=DifferentialEvolution, problem=tcs, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=tcs, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=tcs, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=tcs, N=N, G=G, config=config)

In [11]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
           case1      case2      case3
count  30.000000  30.000000  30.000000
mean    0.012710   0.012749   0.012697
std     0.000054   0.000097   0.000026
min     0.012668   0.012671   0.012667
25%     0.012678   0.012688   0.012680
50%     0.012690   0.012719   0.012690
75%     0.012726   0.012788   0.012704
max     0.012932   0.013174   0.012787

DE
           case1         case2      case3
count  30.000000  3.000000e+01  30.000000
mean    0.012669  1.266546e-02   0.012697
std     0.000004  2.287372e-07   0.000033
min     0.012666  1.266525e-02   0.012667
25%     0.012667  1.266529e-02   0.012676
50%     0.012667  1.266542e-02   0.012689
75%     0.012669  1.266553e-02   0.012699
max     0.012681  1.266634e-02   0.012831

GA
           case1      case2      case3
count  30.000000  30.000000  30.000000
mean    1.159516   1.927843   1.296817
std     2.613368   3.094357   2.792611
min     0.014144   0.019785   0.019785
25%     0.025284   0.056031   0.030230
50%     0.096290   0

# 4.3.4 Example 11: Design of a Speed Reducer

Possuindo 7 dimensões e 11 restrições por meio de desigualdades, o problema é a redução do peso de um redutor de velocidade.

A JADE teve os melhores resultados entre os encontrados em todos os quesitos.

In [16]:
best_x = (3.49999, 0.6999, 17, 7.3, 7.8, 3.3502, 5.2866)
sr = SpeedReducer()
sr.speed_reducer(best_x)-sr.constrains(best_x)

2995.8357571501438

In [ ]:
res_de = run(algorithm=DifferentialEvolution, problem=sr, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=sr, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=sr, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=sr, N=N, G=G, config=config)

In [18]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   3006.315687  3005.805605  3007.473025
std       3.303063     3.550805     3.722529
min    3000.113828  3000.459207  3001.952511
25%    3004.379586  3002.952639  3004.481940
50%    3006.053236  3005.511566  3006.978101
75%    3008.393558  3008.000280  3010.391629
max    3013.346164  3013.271129  3017.480921

DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   2996.211183  2996.204402  2996.208830
std       0.025936     0.000731     0.002079
min    2996.204735  2996.203524  2996.204650
25%    2996.205570  2996.203977  2996.207486
50%    2996.206243  2996.204230  2996.209213
75%    2996.207041  2996.204530  2996.210202
max    2996.348297  2996.206501  2996.213830

GA
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   3539.318137  3719.325917  3314.174285
std     625.920505   867.636690   397.